In [272]:
import pandas as pd
import requests
import json
import numpy as np

In [273]:
ICUBeds = pd.read_csv("ICUsByCounty.csv") 

In [274]:
ICUBeds.head()

,State,County,ICU Beds,Total Population,Population Aged 60+,Percent of Population Aged 60+,Residents Aged 60+ Per Each ICU Bed
0,Alabama,Autauga,6,55036,10523,19.1,1754.0
1,Alabama,Baldwin,51,203360,53519,26.3,1049.0
2,Alabama,Barbour,5,26201,6150,23.5,1230.0
3,Alabama,Bibb,0,22580,4773,21.1,NaN
4,Alabama,Blount,6,57667,13600,23.6,2267.0


In [275]:
api_response = requests.get('https://services1.arcgis.com/Hp6G80Pky0om7QvQ/arcgis/rest/services/Hospitals_1/FeatureServer/0/query?where=1%3D1&outFields=*&outSR=4326&f=json&resultRecordCount=2000')
hospitals = pd.json_normalize(api_response.json()['features'])
df_length = len(hospitals)
api_target = 'https://services1.arcgis.com/Hp6G80Pky0om7QvQ/arcgis/rest/services/Hospitals_1/FeatureServer/0/query?where=1%3D1&outFields=*&outSR=4326&f=json&resultRecordCount=2000'

while True:
    api_response = requests.get(api_target+'&resultOffset='+str(df_length))
    new = pd.json_normalize(api_response.json()['features'])
    hospitals = hospitals.append(new)
    if (df_length == len(hospitals)):
        break
    df_length = len(hospitals)

In [276]:
hospitals = hospitals.rename(columns = {'attributes.BEDS': 'Hospital Beds', 'attributes.COUNTY': 'County'})
hospitals = hospitals.assign(sanitized=lambda df: df['Hospital Beds'].abs())
hospitals['Hospital Beds'] = hospitals['sanitized']
beds = hospitals[['County', 'Hospital Beds']].groupby('County').sum()

In [280]:
beds = beds.assign(lowercase=lambda df: df.index.str.lower())
ICUBeds = ICUBeds.assign(lowercase=lambda df: df.County.str.lower())

In [281]:
result = ICUBeds.merge(beds, how='left', on='lowercase')
result = result.drop('lowercase', axis=1)

In [286]:
result.head()

,State,County,ICU Beds,Total Population,Population Aged 60+,Percent of Population Aged 60+,Residents Aged 60+ Per Each ICU Bed,Hospital Beds
0,Alabama,Autauga,6,55036,10523,19.1,1754.0,85.0
1,Alabama,Baldwin,51,203360,53519,26.3,1049.0,738.0
2,Alabama,Barbour,5,26201,6150,23.5,1230.0,146.0
3,Alabama,Bibb,0,22580,4773,21.1,NaN,1329.0
4,Alabama,Blount,6,57667,13600,23.6,2267.0,511.0


In [283]:
result.to_csv('HospitalAndICUBeds.csv')

In [290]:
covid_deaths = pd.read_csv("covid_deaths_usafacts.csv")
covid_cases = pd.read_csv("covid_confirmed_usafacts.csv") 

In [294]:
covid_cases = covid_cases[['countyFIPS', 'County Name', 'State', 'stateFIPS', '3/24/2020']]
covid_deaths = covid_deaths[['countyFIPS', 'County Name', 'State', 'stateFIPS', '3/24/2020']]

In [296]:
covid_cases = covid_cases.rename(columns = {'3/24/2020': 'Confirmed Cases'})
covid_deaths = covid_deaths.rename(columns = {'3/24/2020': 'Deaths'})

In [297]:
covid_deaths.head()

,countyFIPS,County Name,State,stateFIPS,Deaths
0,0,Statewide Unallocated,AL,1,0
1,1001,Autauga County,AL,1,0
2,1003,Baldwin County,AL,1,0
3,1015,Calhoun County,AL,1,0
4,1017,Chambers County,AL,1,0


In [346]:
covid_cases['County'] = covid_cases['County Name'].str.extract(r'([A-Za-z]+)')
covid_deaths['County'] = covid_deaths['County Name'].str.extract(r'([A-Za-z]+)')

In [347]:
result.head()

,State,County,ICU Beds,Total Population,Population Aged 60+,Percent of Population Aged 60+,Residents Aged 60+ Per Each ICU Bed,Hospital Beds
0,AL,Autauga,6,55036,10523,19.1,1754.0,85.0
1,AL,Baldwin,51,203360,53519,26.3,1049.0,738.0
2,AL,Barbour,5,26201,6150,23.5,1230.0,146.0
3,AL,Bibb,0,22580,4773,21.1,NaN,1329.0
4,AL,Blount,6,57667,13600,23.6,2267.0,511.0


In [348]:
covid_cases.head()

,countyFIPS,County Name,State,stateFIPS,Confirmed Cases,County
0,0,Statewide Unallocated,AL,1,0,Statewide
1,1001,Autauga County,AL,1,1,Autauga
2,1003,Baldwin County,AL,1,4,Baldwin
3,1015,Calhoun County,AL,1,2,Calhoun
4,1017,Chambers County,AL,1,5,Chambers


In [349]:
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'American Samoa': 'AS',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Guam': 'GU',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}

In [350]:
result = result.replace({'State': us_state_abbrev})

In [354]:
temp = result.merge(covid_cases, how='outer', on=['County', 'State'])
covidData = temp.merge(covid_deaths[['State', 'County', 'Deaths']], how='outer', on=['County', 'State'])

In [356]:
covidData.head()

,State,County,ICU Beds,Total Population,Population Aged 60+,Percent of Population Aged 60+,Residents Aged 60+ Per Each ICU Bed,Hospital Beds,countyFIPS,County Name,stateFIPS,Confirmed Cases,Deaths
0,AL,Autauga,6.0,55036.0,10523.0,19.1,1754.0,85.0,1001.0,Autauga County,1.0,1.0,0.0
1,AL,Baldwin,51.0,203360.0,53519.0,26.3,1049.0,738.0,1003.0,Baldwin County,1.0,4.0,0.0
2,AL,Barbour,5.0,26201.0,6150.0,23.5,1230.0,146.0,NaN,NaN,NaN,NaN,NaN
3,AL,Bibb,0.0,22580.0,4773.0,21.1,NaN,1329.0,NaN,NaN,NaN,NaN,NaN
4,AL,Blount,6.0,57667.0,13600.0,23.6,2267.0,511.0,NaN,NaN,NaN,NaN,NaN


In [357]:
covidData = covidData.drop(['Percent of Population Aged 60+', 'Residents Aged 60+ Per Each ICU Bed'], axis=1)

In [359]:
covidData.head(10)

,State,County,ICU Beds,Total Population,Population Aged 60+,Hospital Beds,countyFIPS,County Name,stateFIPS,Confirmed Cases,Deaths
0,AL,Autauga,6.0,55036.0,10523.0,85.0,1001.0,Autauga County,1.0,1.0,0.0
1,AL,Baldwin,51.0,203360.0,53519.0,738.0,1003.0,Baldwin County,1.0,4.0,0.0
2,AL,Barbour,5.0,26201.0,6150.0,146.0,NaN,NaN,NaN,NaN,NaN
3,AL,Bibb,0.0,22580.0,4773.0,1329.0,NaN,NaN,NaN,NaN,NaN
4,AL,Blount,6.0,57667.0,13600.0,511.0,NaN,NaN,NaN,NaN,NaN
5,AL,Bullock,0.0,10478.0,2371.0,61.0,NaN,NaN,NaN,NaN,NaN
6,AL,Butler,7.0,20126.0,5151.0,4554.0,NaN,NaN,NaN,NaN,NaN
7,AL,Calhoun,24.0,115527.0,27115.0,2073.0,1015.0,Calhoun County,1.0,2.0,0.0
8,AL,Chambers,0.0,33895.0,8501.0,154.0,1017.0,Chambers County,1.0,5.0,0.0
9,AL,Cherokee,0.0,25855.0,7513.0,3119.0,NaN,NaN,NaN,NaN,NaN


In [360]:
covidData.to_csv('BedsToCovidByCounty.csv')

In [362]:
covidDataByState = covidData.groupby('State').sum()

In [363]:
covidDataByState.to_csv('BedsToCovidByState.csv')